In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [9]:
class mlp(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(mlp, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, output_dim)
        )

    def forward(self, x):
        return self.network(x)

In [10]:
class rnn(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim, num_layers=1):
        super(rnn, self).__init__()
        self.rnn_h = nn.RNN(input_dim, hidden_dim1, num_layers, nonlinearity='relu', batch_first=True, bidirectional=True)
        #self.fc1 = nn.Linear(24, 18)
        #self.fc2 = nn.Linear(18, 8)
        self.fc3 = nn.Linear(48, output_dim)

    def forward(self, x):
        out, _ = self.rnn_h(x)
        #out = out.squeeze()
        #out = self.fc1(out)
        #out = self.fc2(out)
        out = out.reshape(out.size(dim=0),out.size(dim=1)*out.size(dim=2))
        out = self.fc3(out)
        return out

In [11]:
class TrxEncoder(nn.Module):
    def __init__(self, d_token, n_head, dim_ffn):
        super(TrxEncoder, self).__init__()
        self.encoderLayer = nn.TransformerEncoderLayer(d_token, n_head, dim_ffn, batch_first=True)
        self.encoder = nn.TransformerEncoder(self.encoderLayer, num_layers=3, enable_nested_tensor=False)
        self.ffn = nn.Linear(24,2)
    
    def forward(self, x):
        x = x.reshape(len(x),24,1)
        y = self.encoder(x).squeeze()
        y = self.ffn(y)
        return y

In [12]:
class dataset(Dataset):
    def __init__(self, csv_file):
        df = pd.read_csv(csv_file,header=None)
        N = len(df)
        self.input = torch.tensor(df.iloc[:, :-2].values).float()
        self.input = self.input - 10
        self.output = torch.tensor(df.iloc[:, -2:].values).float()
        #self.output = -self.output
    
    def __len__(self):
        return len(self.output)

    def __getitem__(self, idx):
        return self.input[idx], self.output[idx]

In [13]:
def train(data_loader, model, optimizer, epoch, criterion, device):
    # set the model to training mode
    model.train()
    
    for (step, value) in enumerate(data_loader):
        data = value[0].to(device)
        target = value[1].to(device)
        optimizer.zero_grad()
        pred = model(data)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        
    return loss.item()

In [14]:
def test(data_loader, model, criterion, device):
    # set the model to evaluation mode
    model.eval()
    
    for (step, value) in enumerate(data_loader):
        data = value[0].to(device)
        target = value[1].to(device)
        pred = model(data)
        loss = criterion(pred, target)
        
    return loss.item()

In [15]:
# Check if CUDA is available and set device to GPU if it is
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1337)
torch.cuda.manual_seed(1337)

train_set = dataset("trainset.csv")
test_set = dataset("testset.csv")

input_dim = len(train_set[0][0])
output_dim = len(test_set[0][1])
hidden_dim1 = 18
hidden_dim2 = 8
N = train_set.__len__()

# setup hyperparameters
batch_size = 512
lr = 0.01
n_epoch = 1000

train_loader = DataLoader(
                train_set,
                batch_size=batch_size
            )
test_loader = DataLoader(
                test_set,
                batch_size=batch_size
            )

d_token = 1
n_head = 1
dim_ffn = 24
model = mlp(input_dim, hidden_dim1, hidden_dim2, output_dim)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # Use Adam optimizer
criterion = nn.MSELoss()

# training and evaluation
for ep in range(n_epoch+1):
    train_loss = train(train_loader, model, optimizer, n_epoch, criterion, device)
    if ep%100 == 0:
        test_loss = test(test_loader, model, criterion, device)
        print(f"Epoch {ep}, train_loss: {train_loss}, test_loss: {test_loss}")

Epoch 0, train_loss: 1803.042236328125, test_loss: 1543.71826171875
Epoch 100, train_loss: 10.205170631408691, test_loss: 9.7351713180542
Epoch 200, train_loss: 5.140499591827393, test_loss: 6.028907775878906
Epoch 300, train_loss: 4.630681991577148, test_loss: 5.44679594039917
Epoch 400, train_loss: 4.299884796142578, test_loss: 5.22344446182251
Epoch 500, train_loss: 3.9188232421875, test_loss: 4.954413890838623
Epoch 600, train_loss: 3.4707589149475098, test_loss: 4.826680660247803
Epoch 700, train_loss: 3.2860608100891113, test_loss: 4.327934741973877
Epoch 800, train_loss: 3.054774522781372, test_loss: 4.262598514556885
Epoch 900, train_loss: 3.041782855987549, test_loss: 4.194814682006836
Epoch 1000, train_loss: 2.969710111618042, test_loss: 4.162206172943115


In [111]:
x,y = test_set[:]
x = x.to(device)
y = y.to(device)
print(x)

tensor([[ 3.1849,  3.9992,  1.7387,  ...,  6.4917,  5.4571,  4.1548],
        [ 0.8773, -1.3968, -3.5463,  ...,  7.7472,  6.9451,  6.2222],
        [-3.1082, -1.5100, -4.2988,  ...,  2.9856,  2.2455,  5.3241],
        ...,
        [ 0.0221,  1.0008,  2.0454,  ...,  1.7404,  2.1688,  4.8853],
        [ 3.6667,  1.7543,  2.8841,  ...,  9.0000,  8.1854,  7.5771],
        [-2.8162, -0.7126, -3.6593,  ...,  4.5533,  3.5534,  1.2462]],
       device='cuda:0')


In [112]:
pred = model(x)
print(f"Shape: {pred.shape}")
print(pred[0:10])

Shape: torch.Size([1000, 2])
tensor([[51.1757, 64.6335],
        [53.0023, 62.7058],
        [36.9760, 44.0941],
        [60.2110, 61.5573],
        [76.0446, 76.4686],
        [64.5876, 73.7850],
        [73.2042, 72.6481],
        [57.6305, 64.8977],
        [75.8450, 79.1754],
        [38.8730, 49.6922]], device='cuda:0', grad_fn=<SliceBackward0>)


In [113]:
from sklearn import metrics

# Measure RMSE error.  RMSE is common for regression.

score = np.sqrt(metrics.mean_squared_error(pred.cpu().detach(), y.cpu().detach()))
print(f"Final score (RMSE): {score}")

score = torch.sqrt(torch.nn.functional.mse_loss(pred, y))
print(f"Final score (RMSE) with training dataset: {score}")

Final score (RMSE): 1.9499244689941406
Final score (RMSE) with training dataset: 1.9499242305755615


In [ ]:
# Sample predictions
for i in range(50):
    print(f"{i+1}. Actual: {y[i]}, " + f"Predicted: {pred[i]}")

In [ ]:
train_set[0]

In [ ]:
x,y = train_set[range(4)]
x = x.reshape([4,12,2])
rnn = nn.RNN(2, 1, 1, nonlinearity='relu', batch_first=True)

In [ ]:
print(x)

In [ ]:
y = rnn(x)
print(y)

In [78]:
x = torch.ones([16,24,1])
w = torch.ones([24,18])
z = torch.matmul(torch.t(w),x)
print(z.shape)

torch.Size([16, 18, 1])
